# Predict Future Sale - Baseline
## Final project for "How to win a data science competition" Coursera course
https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data  
>Student: Rafael Caneiro de Oliveira  
>Email: rafael.caneiro@gmail.com  
>Date: 04/08/2020

## Load the data

In [41]:
import numpy as np
import pandas as pd

from pathlib import Path

PATH = Path.cwd().parent
DATA_PATH = Path(PATH, "./data/raw/") 

In [42]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_211"; Java(TM) SE Runtime Environment (build 1.8.0_211-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from /Users/rco/opt/anaconda3/envs/hse_comp_ds/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9v/rrpn016x5n19xkkf9fk88b8m0000gn/T/tmpd_nuxurm
  JVM stdout: /var/folders/9v/rrpn016x5n19xkkf9fk88b8m0000gn/T/tmpd_nuxurm/h2o_rco_started_from_python.out
  JVM stderr: /var/folders/9v/rrpn016x5n19xkkf9fk88b8m0000gn/T/tmpd_nuxurm/h2o_rco_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_rco_vbld1k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [43]:
train_df = h2o.import_file(Path(DATA_PATH,"sales_train.csv").as_uri())
test_df = h2o.import_file(Path(DATA_PATH,"test.csv").as_uri())
items_df = h2o.import_file(Path(DATA_PATH,"items.csv").as_uri())
shops_df = h2o.import_file(Path(DATA_PATH,"shops.csv").as_uri())
categories_df = h2o.import_file(Path(DATA_PATH,"item_categories.csv").as_uri())

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [44]:
train_df.head(5)

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
02.01.2013,0,59,22154,999,1
03.01.2013,0,25,2552,899,1
05.01.2013,0,25,2552,899,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099,1


In [45]:
items_df.head(5)

item_name,item_id,item_category_id
! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",1,76
***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
***КОРОБКА (СТЕКЛО) D,4,40


In [46]:
print(train_df.shape)

train_df = train_df.merge(items_df[["item_id", "item_category_id"]],
                          all_x=True,
                          all_y=False,
                          by_x=["item_id"],
                          by_y=["item_id"])

# train_df = train_df.merge(categories_df,
#                           all_x=True,
#                           all_y=False,
#                           by_x=["item_category_id"],
#                           by_y=["item_category_id"])

# train_df = train_df.merge(shops_df,
#                           all_x=True,
#                           all_y=False,
#                           by_x=["shop_id"],
#                           by_y=["shop_id"])

print(train_df.shape)

train_df["date"] = train_df["date"].as_date("%d.%m.%Y")
train_df["year"] = train_df["date"].year()
train_df["month"] = train_df["date"].month()

categoricals = [
    "shop_id", "item_category_id", "item_id",
]

for col in categoricals:
    train_df[col] = train_df[col].asfactor()

train_df.summary()

(2935849, 6)
(2935849, 7)


,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id,year,month
type,enum,int,int,enum,real,int,enum,int,int
mins,,1356998400000.0,0.0,,-1.0,-22.0,,2013.0,1.0
mean,,1396503874970.6816,14.569911463430167,,890.8532326980118,1.2426408851408905,,2013.7768495586797,6.247716759274743
maxs,,1446249600000.0,33.0,,307980.0,2169.0,,2015.0,12.0
sigma,,24788116176.834805,9.422987708755727,,1729.7996307126589,2.618834430895425,,0.7684789546941898,3.53621934262439
zeros,,0,115690,,0,0,,0,0
missing,0,0,0,0,0,0,0,0,0
0,0,1409529600000.0,20.0,54,58.0,1.0,40,2014.0,9.0
1,1,1396569600000.0,15.0,55,4490.0,1.0,76,2014.0,4.0
2,1,1396396800000.0,15.0,55,4490.0,1.0,76,2014.0,4.0


In [47]:
test_df["date_block_num"] = 34

test_df = test_df.merge(items_df[["item_id", "item_category_id"]],
                        all_x=True,
                        all_y=False,
                        by_x=["item_id"],
                        by_y=["item_id"])

# test_df = test_df.merge(categories_df,
#                         all_x=True,
#                         all_y=False,
#                         by_x=["item_category_id"],
#                         by_y=["item_category_id"])

# test_df = test_df.merge(shops_df,
#                         all_x=True,
#                         all_y=False,
#                         by_x=["shop_id"],
#                         by_y=["shop_id"])

test_df["year"] = 2015
test_df["month"] = 11

for col in categoricals:
    test_df[col] = test_df[col].asfactor()

test_df.summary()

,item_id,ID,shop_id,date_block_num,item_category_id,year,month
type,enum,int,enum,int,enum,int,int
mins,,0.0,,34.0,,2015.0,11.0
mean,,107099.50000000001,,34.0,,2015.0,11.0
maxs,,214199.0,,34.0,,2015.0,11.0
sigma,,61834.35816760776,,0.0,,0.0,0.0
zeros,,1,,0,,0,0
missing,0,0,0,0,0,0,0
0,30,2587.0,5,34.0,40,2015.0,11.0
1,30,7687.0,4,34.0,40,2015.0,11.0
2,30,12787.0,6,34.0,40,2015.0,11.0


## Model

In [48]:
from h2o.automl import H2OAutoML

# Define label and predictors
y = "item_cnt_day"
x = [
    "shop_id", "item_category_id", "item_id", "date_block_num", 
    "year", "month"
]

aml = H2OAutoML(max_runtime_secs=300, max_models=50)
aml.train(x, y, train_df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Evaluation

In [49]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200825_210505,5.81809,2.41207,5.81809,0.32003,nan
StackedEnsemble_BestOfFamily_AutoML_20200825_210505,5.85967,2.42068,5.85967,0.327847,nan
DRF_1_AutoML_20200825_210505,6.13616,2.47713,6.13616,0.315227,nan
GBM_2_AutoML_20200825_210505,6.43498,2.53673,6.43498,0.407291,nan
GBM_4_AutoML_20200825_210505,6.47658,2.54491,6.47658,0.406695,nan
GBM_1_AutoML_20200825_210505,6.53394,2.55616,6.53394,0.416496,nan
GBM_3_AutoML_20200825_210505,6.55081,2.55945,6.55081,0.409306,nan
GBM_5_AutoML_20200825_210505,6.5568,2.56062,6.5568,0.411699,nan
XGBoost_2_AutoML_20200825_210505,6.71407,2.59115,6.71407,0.537014,nan
GLM_1_AutoML_20200825_210505,6.75909,2.59983,6.75909,0.422859,nan


In [ ]:
predictions = aml.predict(test_df[x])
predictions.head(5)

In [ ]:
predictions.as_data_frame()["predict"]

In [ ]:
test_df.head()

In [ ]:
submission = test_df.concat(predictions)
submission["predict"] = submission["predict"].round()
submission[submission["predict"]>20, "predict"] = 20
submission[submission["predict"]<0, "predict"] = 0
submission.summary()

In [ ]:
submission.rename({"predict":"item_cnt_month"})
file = str(Path(PATH, "notebooks/submission.csv"))
h2o.export_file(submission[["ID", "item_cnt_month"]],
                file,
                force=True)

In [ ]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "h2o_automl"